# IMPORTS

In [2]:
import matplotlib
matplotlib.use('Agg')  # Use a non-interactive backend for matplotlib
from matplotlib.backends.backend_pdf import PdfPages
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import boost_histogram as bh


# SETTINGS

In [3]:
# target = 'Carbon'
# run_number_electron = 24242
# beam_charge = 18122.907 # BCM4C Beam Cut Charge: 18122.907 uC
# live_time = 1 # Assumption
# tracking_eff = 1 # Assumption
# prescale_factor = 1 # From report file

# run_number_positron = 24549
# beam_charge_pos = 43022.260 # BCM4C Beam Cut Charge: 43022.260 uC
# live_time_pos = 1 # Assumption
# tracking_eff_pos = 1 # Assumption
# prescale_factor_pos = 3 # From report file

# USE_LOG_SCALE = False
# beam_charge_milliC     = beam_charge / 1000
# beam_charge_milliC_pos = beam_charge_pos / 1000

target = 'Copper'
run_number_electron = 24148
beam_charge = 13614.219 # BCM4C Beam Cut Charge: 13614.219 uC
live_time = 1 # Assumption
tracking_eff = 1 # Assumption
prescale_factor = 1 # From report file

run_number_positron = 24551
beam_charge_pos = 45807.820 # BCM4C Beam Cut Charge: 45807.820 uC
live_time_pos = 1 # Assumption
tracking_eff_pos = 1 # Assumption
prescale_factor_pos = 3 # From report file

USE_LOG_SCALE = False
beam_charge_milliC     = beam_charge / 1000
beam_charge_milliC_pos = beam_charge_pos / 1000

# -----------------------------------------------------
# File paths
# -----------------------------------------------------

electron_file = f"/w/hallc-scshelf2102/c-rsidis/relder/ROOTfiles/hms_coin_replay_production_{run_number_electron}_-1.root"
positron_file = f"/w/hallc-scshelf2102/c-rsidis/relder/ROOTfiles/hms_coin_replay_production_{run_number_positron}_-1.root"

# -----------------------------------------------------
# Defining branches, using uproot to put them in data frames
# -----------------------------------------------------

branches = ["H.gtr.dp", "H.cal.etottracknorm", "H.gtr.ph", "H.gtr.th","H.gtr.x", "H.gtr.y", "H.kin.Q2", "H.kin.x_bj", "H.kin.W", "H.cer.npeSum"]
df_elec = pd.DataFrame(uproot.open(electron_file)["T"].arrays(branches, library="np"))
df_pos  = pd.DataFrame(uproot.open(positron_file)["T"].arrays(branches, library="np"))

# CUTS AND WEIGHTS

In [4]:
cut_elec = (
    df_elec["H.gtr.dp"].between(-8, 8) &
    (df_elec["H.cer.npeSum"] > 2) &
    (df_elec["H.cal.etottracknorm"] > 0.8)
)

cut_pos =(
    df_pos["H.gtr.dp"].between(-8, 8) &
    (df_pos["H.cer.npeSum"] > 2) &
    (df_pos["H.cal.etottracknorm"] > 0.8)
)

df_elec_cut = df_elec[cut_elec].copy()
df_pos_cut  = df_pos[cut_pos].copy()

df_elec_cut["weight"] = prescale_factor / (beam_charge_milliC * tracking_eff * live_time)
df_pos_cut["weight"]  = prescale_factor_pos / (beam_charge_milliC_pos * tracking_eff_pos * live_time_pos)

# VARIABLES AND BINNING

In [5]:
custom_bins = {
    "H.gtr.dp":            dict(num_bins=100,  min=-10, max=10),
    "H.cal.etottracknorm": dict(num_bins=100, min=0.8,   max=1.225),
    "H.gtr.ph":            dict(num_bins=200, min=-0.025,max=0.025),
    "H.gtr.th":            dict(num_bins=100, min=-0.075,max=0.075),
    "H.gtr.x":             dict(num_bins=100, min=-0.5,  max=0.20),
    "H.gtr.y":             dict(num_bins=100, min=-1,  max=1.25),
    "H.kin.Q2":            dict(num_bins=100, min=2.5,   max=4.125),
    "H.kin.x_bj":          dict(num_bins=100, min=0.165,   max=0.335),
    "H.kin.W":             dict(num_bins=100, min=3.1,   max=3.45),
    "H.cer.npeSum":        dict(num_bins=100, min=0,   max=20),
}

wide_bins = {v: dict(num_bins=10000, min=-500, max=500) for v in custom_bins}


# RATIOS, INTEGRALS, AND MAKING THE FIGURES

In [6]:
def compute_ratio_with_uncertainty(hist_num, hist_den):
    num = hist_num.values()
    den = hist_den.values()
    var_num = hist_num.variances()
    var_den = hist_den.variances()
    ratio = np.full_like(num, np.nan, dtype=float)
    ratio_err = np.full_like(num, np.nan, dtype=float)
    valid = (den > 0) & (num > 0)
    ratio[valid] = num[valid] / den[valid]
    rel_err_num = np.zeros_like(num, dtype=float)
    rel_err_den = np.zeros_like(num, dtype=float)
    rel_err_num[valid] = np.sqrt(var_num[valid]) / num[valid]
    rel_err_den[valid] = np.sqrt(var_den[valid]) / den[valid]
    ratio_err[valid] = ratio[valid] * np.sqrt(rel_err_num[valid]**2 + rel_err_den[valid]**2)
    return ratio, ratio_err

# Figures and integrals

def make_comparison_and_ratio_figure(df_elec, df_pos, variable, bin_info, use_log_scale=False):
    axis = bh.axis.Regular(bin_info["num_bins"], bin_info["min"], bin_info["max"])
    bin_centers = axis.centers
    hist_elec = bh.Histogram(axis, storage=bh.storage.Weight())
    hist_pos  = bh.Histogram(axis, storage=bh.storage.Weight())
    hist_elec.fill(df_elec[variable].values, weight=df_elec["weight"].values)
    hist_pos.fill(df_pos[variable].values,  weight=df_pos["weight"].values)
    elec_vals, elec_err = hist_elec.values(), np.sqrt(hist_elec.variances())
    pos_vals,  pos_err  = hist_pos.values(),  np.sqrt(hist_pos.variances())
    ratio_vals, ratio_err = compute_ratio_with_uncertainty(hist_pos, hist_elec)
    # Totals
    elec_total = np.sum(elec_vals)
    pos_total  = np.sum(pos_vals)
    elec_total_err = np.sqrt(np.sum(hist_elec.variances()))
    pos_total_err  = np.sqrt(np.sum(hist_pos.variances()))
    # Figure
    fig, (ax_top, ax_bottom) = plt.subplots(
        nrows=2, ncols=1, figsize=(8, 8),
        sharex=True, gridspec_kw={"height_ratios": [3, 1]}
    )
    ax_top.errorbar(bin_centers, elec_vals, yerr=elec_err, fmt="o",
                    mfc="none", mec="red", ecolor="red", capsize=3, label="Electrons")
    ax_top.errorbar(bin_centers, pos_vals, yerr=pos_err, fmt="o",
                    mfc="none", mec="blue", ecolor="blue", capsize=3, label="Positrons")
    ax_top.set_ylabel("Counts")
    ax_top.set_title(f"{variable} - Counts & Ratio")
    ax_top.grid(alpha=0.5)
    ax_top.legend()
    if use_log_scale:
        ax_top.set_yscale("log")
    ax_bottom.errorbar(bin_centers, ratio_vals, yerr=ratio_err, fmt="o",
                        mfc="none", mec="green", ecolor="green", capsize=3,
                       label="Positron/Electron")
    ax_bottom.axhline(0.1, color="black", linestyle="--", linewidth=1)
    ax_bottom.set_xlabel(variable)
    ax_bottom.set_ylabel("Ratio")
    ax_bottom.grid(alpha=0.5)
    ax_bottom.legend()
    ax_bottom.set_xlim(bin_info["min"], bin_info["max"])
    plt.tight_layout()
    return fig, elec_total, elec_total_err, pos_total, pos_total_err, hist_elec, hist_pos

def save_all_plots(pdf, df_elec, df_pos, bins, use_log_scale):
    for variable, bin_info in bins.items():
        fig, *_ = make_comparison_and_ratio_figure(df_elec, df_pos, variable, bin_info, use_log_scale)
        pdf.savefig(fig)
        fig.clf()
        plt.close(fig)

def compute_integrals_summary(df_elec, df_pos, bins):
    summary = []
    for variable, bin_info in bins.items():
        _, e_tot, e_err, p_tot, p_err, hist_elec, hist_pos = make_comparison_and_ratio_figure(
            df_elec, df_pos, variable, bin_info, use_log_scale=False
        )
        if e_tot > 0 and p_tot > 0:
            ratio_of_totals = p_tot / e_tot
            ratio_of_totals_err = ratio_of_totals * np.sqrt(
                (p_err / p_tot) ** 2 +
                (e_err / e_tot) ** 2
            )
        else:
            ratio_of_totals, ratio_of_totals_err = np.nan, np.nan
        summary.append({
            "Variable": variable,
            "Electron Total": e_tot,
            "Electron Err": e_err,
            "Positron Total": p_tot,
            "Positron Err": p_err,
            "Ratio of Totals": ratio_of_totals,
            "Ratio of Totals Err": ratio_of_totals_err
        })
    return summary

def add_summary_table_pdf(pdf, integrals_summary, target, run_number_electron, run_number_positron):
    fig, ax = plt.subplots(figsize=(12, 7))
    ax.axis("off")
    col_labels = [
        "Variable",
        "Electron Total ± Err",
        "Positron Total ± Err",
        "Ratio of Totals ± Err"
    ]
    table_data = []
    for entry in integrals_summary:
        table_data.append([
            entry["Variable"],
            f"{entry['Electron Total']:.4f} ± {entry['Electron Err']:.4f}",
            f"{entry['Positron Total']:.4f} ± {entry['Positron Err']:.4f}",
            f"{entry['Ratio of Totals']:.8f} ± {entry['Ratio of Totals Err']:.8f}"
                if not np.isnan(entry["Ratio of Totals"]) else "N/A"
        ])
    table = ax.table(cellText=table_data, colLabels=col_labels, loc="center")
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)
    ax.set_title(
        f"Positron Background Comparisons for {target}, e- Runs {run_number_electron} and e+ Runs {run_number_positron}",
        fontsize=14, pad=20
    )
    pdf.savefig(fig)
    fig.clf()
    plt.close(fig)

with PdfPages(f"Positron_Background_{target}.pdf") as pdf:
    # 1. Make plots (custom bins)
    save_all_plots(pdf, df_elec_cut, df_pos_cut, custom_bins, USE_LOG_SCALE)
    # 2. Compute integrals and summary (wide bins)
    integrals_summary = compute_integrals_summary(df_elec_cut, df_pos_cut, wide_bins)
    # 3. Add table page
    add_summary_table_pdf(pdf, integrals_summary, target, run_number_electron, run_number_positron)
print(f"Saved Positron_Background_{target}.pdf with plots and integral summary table including both ratio metrics")


Saved Positron_Background_Copper.pdf with plots and integral summary table including both ratio metrics
